<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/basics/projects.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/basics/projects.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Projects

* A project can be thought of as a specific labeling task on a set of labels
* That set of labels is defined by the data rows attached to the project
* Each project has an ontology which defines the types of annotations supported during the labeling process
**Note that there is a lot of advanced usage that is not covered in this notebook. See examples/project_configuration/project_setup.ipynb for those functions**
* Also note that deprecated functions are not explained here.

In [ ]:
!pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your API key
API_KEY = ""
# To get your API key go to: Workspace settings -> API -> Create API Key
client = lb.Client(api_key=API_KEY)

### Create a project


In [ ]:
# Creates an empty project
project = client.create_project(name="my-test-project",
                                description="a description",
                                media_type=lb.MediaType.Image)

### Create a dataset with data rows

In [ ]:
dataset = client.create_dataset(name="project-demo-dataset")
global_keys = []
uploads = []
# Generate data rows
for i in range(1,9):
    gb_key = "TEST-ID-%id" % uuid.uuid1()
    uploads.append({
        'row_data':  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": gb_key,
    })
    global_keys.append(gb_key)

task = dataset.create_data_rows(uploads)
task.wait_till_done()
print("ERRORS: " , task.errors)
print("RESULT URL: ", task.result_url)

### Add a data rows to a project 


In [ ]:
project.create_batch(
  "project-demo", # each batch in a project must have a unique name
  global_keys=global_keys,  # paginated collection of data row objects, list of data row ids or global keys
  priority=1 # priority between 1(highest) - 5(lowest)
)

### Attach ontology and label data rows

In this section, we are creating an ontology to attach to a project and creating labels to import as ground truths. We need this setup to demonstrate other methods later in the demo. For more information, please reference our [Ontology](https://docs.labelbox.com/reference/ontology) and [Import Image Annotation](https://docs.labelbox.com/reference/import-image-annotations) development guides.

Create your ontology

In [ ]:
# Create normalized json with a radio classification
ontology_builder = lb.OntologyBuilder(classifications=[  # List of Classification objects
        lb.Classification(class_type=lb.Classification.Type.RADIO,
                          name="radio_question",
                          options=[
                              lb.Option(value="first_radio_answer"),
                              lb.Option(value="second_radio_answer")
                          ]),
])
# Creating an ontology
ontology = client.create_ontology("test-ontology",
                                  ontology_builder.asdict())

Attach ontology to project

In [ ]:

project.setup_editor(ontology)

Create labels and upload them to project as ground truths

In [ ]:
# Create labels
labels = []
for global_key in global_keys:
    labels.append(lb_types.Label(data=lb_types.ImageData(global_key=global_key),
                   annotations=[
                        # Create radio classification annotation for labels
                        lb_types.ClassificationAnnotation(
                         name="radio_question",
                         value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
                         name="second_radio_answer")))
                   ]))

# Upload labels for the data rows in project
upload_job = lb.LabelImport.create_from_objects(
     client = client,
     project_id = project.uid,
     name="label_import_job"+str(uuid.uuid4()),
     labels=labels)

upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")

### Move data rows in project to different task queues

In [ ]:
# Get list of task queues for project
task_queues = project.task_queues()

for task_queue in task_queues:
    print(task_queue)

In [ ]:
project.move_data_rows_to_task_queue(data_row_ids=lb.GlobalKeys(global_keys), #Provide a list of global keys
                                     task_queue_id=task_queues[2].uid #Passing None moves data rows to "Done" task queue
                                    )

### Fetch project configuration

In [ ]:
# Note the project is not fully setup many of the fields will be empty. 
print("Project is not setup yet:", project.setup_complete is None)
print("Project name:", project.name)
print("Project description:", project.description)
print("Media Type:", project.media_type)
batches = [b for b in project.batches()]
print("Project Batches", batches)
print("Ontology:", project.ontology())

### Clean Up

In [ ]:
# project.delete()
# dataset.delete()
# client.delete_unused_ontology(ontology.uid)